In [1]:
import pandas as pd

In [2]:
deliveries = pd.read_csv('../data/raw/deliveries.csv')

In [3]:
deliveries.head()

,id,route_id,state,_status,items,total_size,pickupLat,pickupLng,deliveryLat,deliveryLng,...,delivery_arrival_dt,delivery_effective_dt,cost,final_price,distance,shipper_id,distance_charged,distance_route,total_package_price,vehicle_capacity
0,t-amabrt-1743660,r-shiinc-1248026,7,completed,"[{""id"":""SFYBR004818881"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.4522,-44.2543,...,2022-05-02 15:49:13,2022-05-02 15:52:13,4.47,5.76,72.2,247379.0,0.3,0.3,19.72,2
1,t-amabrt-1743661,r-shiinc-1248060,7,completed,"[{""id"":""SFYBR004812869"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.8932,-43.9991,...,2022-05-02 17:14:51,2022-05-02 17:16:29,2.99,6.87,12.2,222936.0,0.2,0.2,30.04,2
2,t-amabrt-1743662,r-shiinc-1248042,7,completed,"[{""id"":""SFYBR004809553"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.8820,-44.0004,...,2022-05-02 19:13:07,2022-05-02 19:14:19,2.99,4.87,13.3,348598.0,0.4,0.4,39.60,2
3,t-amabrt-1743663,r-shiinc-1248032,7,completed,"[{""id"":""SFYBR004808848"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.7904,-43.9844,...,2022-05-02 20:52:14,2022-05-02 20:56:31,2.99,4.80,27.7,393179.0,0.9,0.9,30.63,2
4,t-amabrt-1743664,r-shiinc-1248067,7,completed,"[{""id"":""SFYBR004808369"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.8798,-43.9029,...,2022-05-02 17:21:59,2022-05-02 17:27:24,2.99,5.94,18.9,269174.0,0.0,0.0,4.90,2


In [4]:
routes_ids_completed=deliveries.route_id.loc[deliveries.state == 7].unique()

In [5]:
routes_totally_complete=[]
for i in routes_ids_completed:
  routes_status=deliveries._status[deliveries.route_id==i].unique()
  if routes_status.size==1 and routes_status[0]=='completed':
    routes_totally_complete.append(i)

In [10]:
deliveries = deliveries.loc[deliveries.route_id.isin( routes_totally_complete)]

In [11]:
deliveries.shape

(85161, 26)

In [12]:
date_fields = [
  "pickup_dt", "max_pickup_dt", "pickup_effective_dt", 
  "delivery_dt", "max_delivery_dt", "delivery_effective_dt"
]
for field in date_fields:
  deliveries = deliveries.loc[deliveries[field] != "0000-00-00 00:00:00"]
  deliveries[field] = pd.to_datetime(deliveries[field])
deliveries.dropna(subset=date_fields, inplace=True)
deliveries.shape

(85161, 26)

In [13]:
deliveries.isna().sum()

id                          0
route_id                    0
state                       0
_status                     0
items                       0
total_size                  0
pickupLat                   0
pickupLng                   0
deliveryLat                 0
deliveryLng                 0
pickup_dt                   0
max_pickup_dt               0
pickup_arrival_dt           0
pickup_effective_dt         0
delivery_dt                 0
max_delivery_dt             0
delivery_arrival_dt        63
delivery_effective_dt       0
cost                        0
final_price                 0
distance                    0
shipper_id                  0
distance_charged            0
distance_route              0
total_package_price      9940
vehicle_capacity            0
dtype: int64

In [14]:
steps = pd.read_csv('../data/raw/steps.csv')

In [46]:
steps.drop_duplicates(inplace=True)

In [15]:
steps.head()

,route_id,delivery_id,action,step_order
0,r-shiinc-1247999,t-amabrt-1743687,delivery,66
1,r-shiinc-1247999,t-amabrt-1743687,pickup,32
2,r-shiinc-1247999,t-amabrt-1743737,delivery,69
3,r-shiinc-1247999,t-amabrt-1743737,pickup,10
4,r-shiinc-1247999,t-amabrt-1743812,delivery,65


In [47]:
(steps.groupby('route_id')["delivery_id"].count() % 2 != 0).sum()

0

In [48]:
steps.groupby('route_id')["delivery_id"].count()

route_id
r-shiinc-1246958     22
r-shiinc-1246971     34
r-shiinc-1247025     24
r-shiinc-1247028     36
r-shiinc-1247029     18
                   ... 
r-shiinc-1318979    104
r-shiinc-1318988    116
r-shiinc-1318998    124
r-shiinc-1319014    110
r-shiinc-1320806     74
Name: delivery_id, Length: 2972, dtype: int64

In [49]:
df_grouped_routes_steps = (steps.groupby('route_id')["delivery_id"].count() / 2).reset_index()
df_grouped_routes_steps.columns = ['route_id', 'deliveries_count']
df_grouped_routes_steps['deliveries_count'] = df_grouped_routes_steps['deliveries_count'].astype(int)

In [50]:
df_grouped_routes_deliveries = deliveries.groupby('route_id')["id"].count().reset_index()
df_grouped_routes_deliveries.columns = ['route_id', 'deliveries_count']
df_grouped_routes_deliveries['deliveries_count'] = df_grouped_routes_deliveries['deliveries_count'].astype(int)


In [41]:
df_grouped_routes_steps

,route_id,deliveries_count
0,r-shiinc-1246958,22
1,r-shiinc-1246971,34
2,r-shiinc-1247025,24
3,r-shiinc-1247028,36
4,r-shiinc-1247029,18
...,...,...
2967,r-shiinc-1318979,416
2968,r-shiinc-1318988,116
2969,r-shiinc-1318998,186
2970,r-shiinc-1319014,55


In [40]:
df_grouped_routes_deliveries

,route_id,deliveries_count
0,r-shiinc-1246958,8
1,r-shiinc-1246971,14
2,r-shiinc-1247025,10
3,r-shiinc-1247029,8
4,r-shiinc-1247030,10
...,...,...
2556,r-shiinc-1318979,1
2557,r-shiinc-1318988,1
2558,r-shiinc-1318998,1
2559,r-shiinc-1319014,1


In [51]:
routes_ids = df_grouped_routes_steps.merge(df_grouped_routes_deliveries, on=['route_id', 'deliveries_count'])['route_id'].unique()

In [57]:
deliveries = deliveries.loc[deliveries.route_id.isin(routes_ids)]

In [58]:
deliveries.shape

(44115, 26)

In [61]:
deliveries.reset_index(drop=True, inplace=True)

In [63]:
steps

,route_id,delivery_id,action,step_order
0,r-shiinc-1247999,t-amabrt-1743687,delivery,66
1,r-shiinc-1247999,t-amabrt-1743687,pickup,32
2,r-shiinc-1247999,t-amabrt-1743737,delivery,69
3,r-shiinc-1247999,t-amabrt-1743737,pickup,10
4,r-shiinc-1247999,t-amabrt-1743812,delivery,65
...,...,...,...,...
1551035,r-shiinc-1319014,t-amabrt-1863361,delivery,106
1552008,r-shiinc-1319014,t-amabrt-1863277,delivery,107
1552981,r-shiinc-1319014,t-amabrt-1863289,delivery,108
1553929,r-shiinc-1319014,t-amabrt-1863288,delivery,109


In [71]:
deliveries = deliveries.merge(steps.loc[steps.action.str.contains('pickup')][["delivery_id", "step_order"]], left_on='id', right_on='delivery_id', how='left')
deliveries = deliveries.merge(steps.loc[steps.action.str.contains('delivery')][["delivery_id", "step_order"]], left_on='id', right_on='delivery_id', how='left')

In [73]:
deliveries["pickup_step_order"] = deliveries["step_order_x"]
deliveries["delivery_step_order"] = deliveries["step_order_y"]
deliveries.drop(columns=["step_order_x", "step_order_y", "delivery_id_x", "delivery_id_y"], inplace=True)

In [79]:
deliveries.head()

,id,route_id,state,_status,items,total_size,pickupLat,pickupLng,deliveryLat,deliveryLng,...,cost,final_price,distance,shipper_id,distance_charged,distance_route,total_package_price,vehicle_capacity,pickup_step_order,delivery_step_order
0,t-amabrt-1743677,r-shiinc-1248030,7,completed,"[{""id"":""SFYBR004808361"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.9327,-43.9440,...,2.99,4.95,11.1,110668.0,0.2,0.2,49.89,2,21,131
1,t-amabrt-1743713,r-shiinc-1248030,7,completed,"[{""id"":""SFYBR004823678"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.9372,-43.9388,...,2.99,4.87,11.9,110668.0,0.2,0.2,16.00,2,33,86
2,t-amabrt-1743749,r-shiinc-1248000,7,completed,"[{""id"":""SFYBR004818385"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.4675,-44.2061,...,5.57,5.94,74.5,174894.0,0.1,0.1,31.58,2,48,76
3,t-amabrt-1743780,r-shiinc-1248349,7,completed,"[{""id"":""SFYBR004822031"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.8953,-43.8016,...,2.99,5.84,34.6,354350.0,0.3,0.3,19.99,2,49,96
4,t-amabrt-1743784,r-shiinc-1248030,7,completed,"[{""id"":""SFYBR004825735"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.9338,-43.9436,...,2.99,4.87,11.1,110668.0,0.5,0.5,24.76,2,20,122


In [80]:
deliveries.to_csv('../data/deliveries.csv', index=False)